In [14]:
%matplotlib inline
import proj1_helpers as utils
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload
import algorithms as ML_alg
import preprocessing_functions as prf

(y, tx, event_ids) = utils.load_csv_data("../train.csv")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
(tx_clean, y_clean) = prf.get_clean_data(tx,y)

y_clean_bin = prf.pass_data_to_zero_one(y_clean).reshape([y_clean.shape[0],1])


std_tx_clean = prf.standardize(tx_clean)

In [95]:
std_tx_clean.shape


(68114, 30)

In [111]:
from matplotlib.mlab import PCA
pca = PCA(tx_clean)
pca.Y.shape

final_tx = ML_alg.build_poly(x = pca.Y, degree=3)
final_tx.shape

(68114, 91)

In [ ]:
initial_w = np.zeros([91,1])




(w, loss ) = ML_alg.gradient_descent(tx = final_tx, y = y_clean_bin, initial_w = initial_w,cost = 'reg_logistic',
                        lambda_= 0.5, gamma = 0.00000001, update_gamma= False, max_iters = 10000)

In [116]:
def evaluate(tx,y,w):
    true = 0
    false = 0
    for i in range(y.shape[0]):
        res = tx[i].dot(w.reshape([91,1]))
        if(abs(1-res) < abs(res)):
            if (y[i] == 1):
                true += 1
            else:
                false += 1
        if(abs(1-res) > abs(res)):
            if (y[i] == 0):
                true += 1
            else:
                false += 1
    return (true,false)

In [134]:
(true, false) = evaluate(final_tx, y_clean_bin, w)

print(true,false)

41125 26989
